llamaindexを用いたragシステム

In [ ]:
title = "NoGame"

In [ ]:
from tqdm.notebook import tqdm
from llama_index import ServiceContext, VectorStoreIndex
from llama_index.schema import TextNode

In [ ]:
import os
import openai

# 環境変数の設定
os.environ["OPENAI_API_KEY"] = "-------伏字-------"
openai.api_key = os.environ["OPENAI_API_KEY"]

In [ ]:
import json

with open(r"RAG\NoGame_summary_corpus.json","r") as f:
    corpus_ = json.load(f)
print(corpus_)

In [ ]:
serihus = []

with open("row_anime_voices\scenes_dialogue_re_token.txt","r") as f:
    t = f.read()
    serihus = t.split("[SEP]")


In [ ]:
import json

with open(r"RAG\NoGame_summary_corpus.json","r") as f:
    json_object = json.load(f)

serihus_json = {}

count = 0
print(len(json_object))
for key in json_object:
    serihus_json[key] = serihus[count]
    count += 1

In [ ]:
with open(r"RAG\NoGame_summary_serihu_corpus.json","w",encoding="utf-8") as f:
    json.dump(serihus_json,f,ensure_ascii=False)

In [ ]:
with open(r"RAG\NoGame_summary_serihu_corpus.json","r") as f:
    serihus_json = json.load(f)

In [ ]:
print(serihus_json)
print(corpus_)

In [ ]:
def RAG(scenario_id):
    serihu = serihus_json[scenario_id].split("\n")
    serihu_character = []
    for line in serihu:
        sep = line.split(",")
        if (len(sep) == 2):        
            if (sep[1] == "ステフ"):
                serihu_character.append(sep[0])    
    return serihu_character
    

In [ ]:
def call_finetuned_model(text,finetuned_gpt_model,character,scenario,serihu):
    completion = openai.ChatCompletion.create(
    # ここに学習済みモデルの model_name を入れる
    model=finetuned_gpt_model,
    messages=[
        {"role": "system", "content": f"""
        あなたはノーゲームノーライフという作品の{character}になりきって回答してください。
        あなたは{character}の口調、語彙、マナーで回答してください。
        説明は書かないでください。
        ただ、{character}といての回答のみを行ってください。
        あなたは{character}に関する知識をすべて知っています。
        また、私がゲームマスターとして必要な知識をささやくので、そちらを考慮してください。
         
        ゲームマスター:「以下は作品内のあるシーンでの出来事(シナリオ)である。回答の参考にせよ」
        ---------
        {scenario}
        ---------
         
        ゲームマスター:「以下は、先ほど紹介した作品内のシーンで、{character}が行った発話である。会話が不自然にならなければ再利用せよ。」
        --------
        {serihu}
        --------

        ゲームマスター:「回答はユーザーが退屈しないようにシナリオを参考に20字程度で答えろ」

         """},{"role": "user", "content": text}
    ]
    )

    return completion.choices[0].message['content']

In [ ]:
def search(query,embed_model):
        service_context = ServiceContext.from_defaults(embed_model=embed_model)
        nodes = [TextNode(id_=id_, text=text) for id_, text in corpus.items()] 
        index = VectorStoreIndex(
        nodes, 
        service_context=service_context, 
        show_progress=True
        )
        retriever = index.as_retriever(similarity_top_k=1)
        retrieved_nodes = retriever.retrieve(query)
        retrieved_ids = [node.node.node_id for node in retrieved_nodes]
        return retrieved_ids[0]

finetuned = f"local:{title}_exp_finetune"
search("空がクラミーの戴冠式に乗り込むとき、ステフはどう思た。",finetuned)

In [ ]:
def chat_character(text,character,corpus_,embed_finetuned_model_,finetuned_gpt_model_):
    scenario_id = search(text,embed_finetuned_model_)
    scenario_text = corpus_[scenario_id]
    serihu = RAG(scenario_id)
    answer = call_finetuned_model(text,finetuned_gpt_model_,character,scenario_text,serihu)
    return answer,serihu,scenario_text
    

In [ ]:
embed_finetuned_model = f"local:{title}_summary_exp_finetune"
finetuned_gpt_model = "ft:gpt-3.5-turbo-0613:personal::85pOJKua"
user_input = input()


print(chat_character(user_input,"ステフ",corpus_,embed_finetuned_model,finetuned_gpt_model))